In [1]:
import keras
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
raw_data = pd.read_csv('C:/Users/young/Desktop/R/download/004170.csv', thousands=',')

In [3]:
raw_data[:5]

,date,open,high,low,close,vol,per,pbr
0,2006-01-02,290493,290493,287210,288523,79190,3790,689
1,2006-01-03,288852,300012,287539,299684,53337,3937,716
2,2006-01-04,301981,322004,299684,318722,105611,4187,761
3,2006-01-05,321019,322332,314126,320034,59538,4204,765
4,2006-01-06,321676,322332,313798,322332,45643,4234,770


In [4]:
data = pd.concat([raw_data['open'],raw_data['vol'],raw_data['per'],raw_data['pbr']],axis=1)

In [5]:
moving = pd.Series()
for cat in data.columns:
    for i in (5, 10, 30, 60):
        moving = pd.concat([moving, data[cat].rolling(i).mean()], axis=1)

In [6]:
moving.columns = ['0', 'moving_p5','moving_p10','moving_p30','moving_p60',
                  'moving_vol5','moving_vol10','moving_vol30','moving_vol60',
                  'moving_per5','moving_per10','moving_per30','moving_per60',
                  'moving_pbr5','moving_pbr10','moving_pbr30','moving_pbr60']

In [7]:
moving = pd.concat([moving, data.open], axis=1)

In [8]:
moving = moving[moving.columns[1:]]

In [9]:
moving = moving[61:]

In [10]:
rn = round(len(moving)*0.7)
train = moving[:rn]
test = moving[rn+1:]

In [11]:
train = (train-np.mean(train)) / np.std(train)
test = (test-np.mean(test)) / np.std(test)

In [12]:
train_x = train[train.columns[:-1]]
train_y = train['open']

test_x = test[test.columns[:-1]]
test_y = test['open']

In [13]:
lag = 30
trx = []

for i in range(len(train_x)-lag):
    trx.append(np.reshape(np.array(train_x[i:i+lag]), (lag,16)))
train_x = np.reshape(trx, (len(trx), lag, 16))

train_y = train_y[lag:]

In [14]:
tex = []

for i in range(len(test_x)-lag):
    tex.append(np.reshape(np.array(test_x[i:i+lag]), (lag,16)))
test_x = np.reshape(tex, (len(tex), lag, 16))

test_y = test_y[lag:]

In [33]:
from keras import models, layers

model = models.Sequential()

model.add(layers.GRU(128, input_shape=(lag,16), return_sequences=True, activation='relu'))
model.add(layers.GRU(128, return_sequences=False, activation='relu'))
model.add(layers.Dense(1))

model.compile(optimizer = 'adam',
             loss = 'mse',
             metrics=['mae'])

In [35]:
from keras import models, layers

model = models.Sequential()

model.add(layers.Conv1D(32, kernel_size=3, input_shape=(lag,16), activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling1D())
model.add(layers.Conv1D(64, kernel_size=3, activation='relu'))
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dense(1))

model.compile(optimizer = 'adam',
             loss = 'mse',
             metrics=['mae'])

In [31]:
from keras import models, layers

model = models.Sequential()

model.add(layers.GRU(128, input_shape=(30,16), return_sequences=True, activation='relu'))
model.add(layers.Bidirectional(layers.GRU(128, activation='relu', return_sequences=True)))
model.add(layers.GRU(128, return_sequences=False))
model.add(layers.Dense(1))

model.compile(optimizer = 'adam',
             loss = 'mse',
             metrics=['mae'])

In [36]:
histroy = model.fit(train_x, train_y,
                   epochs=10,
                    batch_size=512,
                    validation_split=0.2
                   )

Train on 1790 samples, validate on 448 samples
Epoch 1/10
1790/1790 [==============================] - 1s 727us/step - loss: 0.4511 - mean_absolute_error: 0.5485 - val_loss: 0.5713 - val_mean_absolute_error: 0.6783
Epoch 2/10
1790/1790 [==============================] - 0s 18us/step - loss: 0.1721 - mean_absolute_error: 0.3414 - val_loss: 0.1087 - val_mean_absolute_error: 0.2591
Epoch 3/10
1790/1790 [==============================] - 0s 26us/step - loss: 0.1277 - mean_absolute_error: 0.2767 - val_loss: 0.0823 - val_mean_absolute_error: 0.2254
Epoch 4/10
1790/1790 [==============================] - 0s 24us/step - loss: 0.0919 - mean_absolute_error: 0.2359 - val_loss: 0.1436 - val_mean_absolute_error: 0.3166
Epoch 5/10
1790/1790 [==============================] - 0s 26us/step - loss: 0.0921 - mean_absolute_error: 0.2379 - val_loss: 0.0631 - val_mean_absolute_error: 0.2006
Epoch 6/10
1790/1790 [==============================] - 0s 26us/step - loss: 0.0823 - mean_absolute_error: 0.2224 - v